In [3]:
# Jupyter Notebook Simulation Code
# Paste this into a single cell and run it.
# Assumes:
# - You're running this in a Jupyter notebook located in the same directory as aws_scanner_service.py
# - The project structure is intact (e.g., scanner/ and run_parallel_scanner/ modules are accessible)
# - All dependencies (pandas, asyncio, etc.) are installed
# - This simulates Monday 2025-09-22 00:01:00 UTC (a date where ALL timeframes 1d/2d/3d/4d/1w are active)
#   This corresponds closely to Sunday midnight UTC+1 (23:01 UTC Sunday 2025-09-21, but adjusted to trigger 1w on Monday UTC)

import asyncio
import sys
import os
import logging
import signal
from datetime import datetime, timedelta, time
import time as time_module
import pandas as pd

# Clear any existing cache
try:
    from scanner.main import kline_cache
    kline_cache.clear()
except ImportError:
    print("Warning: Could not import kline_cache - ensure project path is set correctly")

# Set project paths (adjust if your notebook is not in the script's directory)
current_dir = os.getcwd()
project_root = os.path.dirname(current_dir)  # Assume notebook is in the script's subdir
sys.path.insert(0, project_root)
sys.path.insert(0, current_dir)

# Configure logging for Jupyter (outputs to notebook)
def setup_logging(debug_mode=False):
    level = logging.DEBUG if debug_mode else logging.INFO
    
    logs_dir = os.path.join(current_dir, "logs")
    os.makedirs(logs_dir, exist_ok=True)
    
    log_file = os.path.join(logs_dir, "scanner_service.log")
    
    logging.basicConfig(
        level=level,
        format='%(asctime)s - %(levelname)s: %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        handlers=[
            logging.StreamHandler(sys.stdout),
            logging.FileHandler(log_file)
        ]
    )
    
    logging.getLogger('telegram').setLevel(logging.INFO)
    logging.getLogger('httpx').setLevel(logging.WARNING)
    logging.getLogger('httpcore').setLevel(logging.WARNING)
    
    return logging.getLogger("ScannerService")

logger = setup_logging(debug_mode=False)  # Set debug_mode=True for more verbose output

# ═════════════════════════════════════════════════════════════════════════════════════════
# Exchange categorization with fast/slow classification
# ═════════════════════════════════════════════════════════════════════════════════════════

# Fast exchanges (reliable, fast API responses)
fast_spot_exchanges = [
    "binance_spot",
    "bybit_spot",
    "gateio_spot"
]

fast_futures_exchanges = [
    "binance_futures",
    "bybit_futures", 
    "gateio_futures"
]

# Slow exchanges (slower API responses, need careful rate limiting)
slow_spot_exchanges = [
    "kucoin_spot",
    "mexc_spot"
]

slow_futures_exchanges = [
    "mexc_futures"
]

# All exchanges grouped by type
all_fast_exchanges = fast_spot_exchanges + fast_futures_exchanges
all_slow_exchanges = slow_spot_exchanges + slow_futures_exchanges
all_spot_exchanges = fast_spot_exchanges + slow_spot_exchanges
all_futures_exchanges = fast_futures_exchanges + slow_futures_exchanges

# ═════════════════════════════════════════════════════════════════════════════════════════
# Strategy configurations by type and priority
# ═════════════════════════════════════════════════════════════════════════════════════════

# Strategy classification
native_strategies = [
    "confluence", "consolidation_breakout", "channel_breakout", 
    "loaded_bar", "trend_breakout", "pin_up", "sma50_breakout"
]

composed_strategies = [
    "hbs_breakout", "vs_wakeup"
]

futures_only_strategies = [
    "reversal_bar", "pin_down"
]

# All timeframes to scan
all_timeframes = ["1d", "2d", "3d", "4d", "1w"]

# Main scan configurations - organized by priority and strategy type
scan_configs = [
    # ────────────────────────────────────────────────────────────────────────────────────
    # PRIORITY 1: FAST EXCHANGES - NATIVE STRATEGIES (highest priority for DB development)
    # ────────────────────────────────────────────────────────────────────────────────────
    {
        "name": "fast_native_strategies",
        "timeframes": all_timeframes,
        "strategies": native_strategies,
        "exchanges": fast_spot_exchanges + ["binance_futures"],
        "users": ["default", "user1", "user2"],
        "send_telegram": True,
        "min_volume_usd": None,
        "priority": 1,
        "exchange_type": "fast_mixed",
        "strategy_type": "native"
    },
    
    # ────────────────────────────────────────────────────────────────────────────────────
    # PRIORITY 2: FAST EXCHANGES - COMPOSED STRATEGIES  
    # ────────────────────────────────────────────────────────────────────────────────────
    {
        "name": "fast_composed_strategies",
        "timeframes": all_timeframes,
        "strategies": composed_strategies,
        "exchanges": fast_spot_exchanges + ["binance_futures"],
        "users": ["default", "user1", "user2"],
        "send_telegram": True,
        "min_volume_usd": None,
        "priority": 2,
        "exchange_type": "fast_mixed",
        "strategy_type": "composed"
    },
    
    # ────────────────────────────────────────────────────────────────────────────────────
    # PRIORITY 3: FAST FUTURES - FUTURES-ONLY STRATEGIES
    # ────────────────────────────────────────────────────────────────────────────────────
    {
        "name": "fast_futures_only",
        "timeframes": all_timeframes,
        "strategies": futures_only_strategies,
        "exchanges": fast_futures_exchanges,
        "users": ["default"],
        "send_telegram": True,
        "min_volume_usd": None,
        "priority": 3,
        "exchange_type": "fast_futures",
        "strategy_type": "futures_only"
    },
    
    # ────────────────────────────────────────────────────────────────────────────────────
    # PRIORITY 4: SLOW SPOT - NATIVE STRATEGIES
    # ────────────────────────────────────────────────────────────────────────────────────
    {
        "name": "slow_native_strategies",
        "timeframes": all_timeframes,
        "strategies": native_strategies,
        "exchanges": slow_spot_exchanges,
        "users": ["default", "user1", "user2"],
        "send_telegram": True,
        "min_volume_usd": None,
        "priority": 4,
        "exchange_type": "slow_spot",
        "strategy_type": "native"
    },
    
    # ────────────────────────────────────────────────────────────────────────────────────
    # PRIORITY 5: SLOW SPOT - COMPOSED STRATEGIES
    # ────────────────────────────────────────────────────────────────────────────────────
    {
        "name": "slow_composed_strategies", 
        "timeframes": all_timeframes,
        "strategies": composed_strategies,
        "exchanges": slow_spot_exchanges,
        "users": ["default", "user1", "user2"],
        "send_telegram": True,
        "min_volume_usd": None,
        "priority": 5,
        "exchange_type": "slow_spot",
        "strategy_type": "composed"
    }
    
    # Note: slow_futures_exchanges reserved for future use
    # Can be added when needed with priority 6+
]

# ═════════════════════════════════════════════════════════════════════════════════════════
# Optimized timeframe scheduling and cache management
# ═════════════════════════════════════════════════════════════════════════════════════════

def get_aggregated_timeframes():
    """Return timeframes that require daily data aggregation"""
    return ["2d", "3d", "4d"]

def get_native_timeframes():
    """Return timeframes with native exchange support"""
    return ["1d", "1w"]

def should_clear_cache_for_session(timeframes):
    """
    Determine if cache should be cleared after a session.
    Clear cache after any session containing aggregated timeframes.
    """
    aggregated_tfs = get_aggregated_timeframes()
    return any(tf in aggregated_tfs for tf in timeframes)

class OptimizedSessionManager:
    """Manages efficient data fetching and cache for aggregated timeframes"""
    
    def __init__(self):
        self.session_data_cache = {}  # Cache for 1d data used across aggregated timeframes
        
    def clear_session_cache(self):
        """Clear the session-level cache (daily data for aggregation)"""
        self.session_data_cache.clear()
        logger.info("Session-level cache cleared")
        
    def needs_daily_data(self, timeframes):
        """Check if any timeframe in the list needs daily data for aggregation"""
        return any(tf in get_aggregated_timeframes() for tf in timeframes)
        
    async def prepare_session_data(self, timeframes, exchanges):
        """
        Pre-fetch daily data once if needed for aggregated timeframes.
        This avoids multiple API calls for the same 1d data.
        """
        if not self.needs_daily_data(timeframes):
            return  # No aggregated timeframes, no prep needed
            
        logger.info("Pre-fetching daily data for aggregated timeframes optimization")
        
        # We don't actually pre-fetch here as the parallel scanner handles this
        # This is a placeholder for future optimization where we could pre-warm cache
        # with a single daily fetch per exchange before running aggregated timeframes
        pass

# Global session manager
session_manager = OptimizedSessionManager()

# ═════════════════════════════════════════════════════════════════════════════════════════
# Enhanced scheduling logic
# ═════════════════════════════════════════════════════════════════════════════════════════

def get_next_candle_time(interval="1d", now=None):
    """
    Calculate time until next candle close for a given interval
    
    Args:
        interval (str): Timeframe interval ('1d', '2d', '3d', '4d', '1w')
        now (datetime, optional): Mock current time for testing
        
    Returns:
        datetime: Next candle close time in UTC
    """
    now = now or datetime.utcnow()
    
    if interval == "1d":
        next_time = now.replace(hour=0, minute=1, second=0, microsecond=0)
        if now >= next_time:
            next_time += timedelta(days=1)
    
    elif interval == "2d":
        reference_date = pd.Timestamp('2025-03-20').normalize()
        today = pd.Timestamp(now.date())
        days_diff = (today - reference_date).days
        period = days_diff // 2
        next_period_start = reference_date + timedelta(days=period * 2 + 2)
        next_time = datetime.combine(next_period_start, time(0, 1, 0))
        if now >= next_time:
            next_time = datetime.combine(next_period_start + timedelta(days=2), time(0, 1, 0))
        while next_time <= now:
            next_time += timedelta(days=2)
    
    elif interval == "3d":
        reference_date = pd.Timestamp('2025-03-20').normalize()
        today = pd.Timestamp(now.date())
        days_diff = (today - reference_date).days
        period = days_diff // 3
        next_period_start = reference_date + timedelta(days=period * 3 + 3)
        next_time = datetime.combine(next_period_start, time(0, 1, 0))
        if now >= next_time:
            next_time = datetime.combine(next_period_start + timedelta(days=3), time(0, 1, 0))
        while next_time <= now:
            next_time += timedelta(days=3)
    
    elif interval == "4d":
        reference_date = pd.Timestamp('2025-03-22').normalize()
        today = pd.Timestamp(now.date())
        days_diff = (today - reference_date).days
        period = days_diff // 4
        next_period_start = reference_date + timedelta(days=period * 4 + 4)
        next_time = datetime.combine(next_period_start, time(0, 1, 0))
        if now >= next_time:
            next_time = datetime.combine(next_period_start + timedelta(days=4), time(0, 1, 0))
        while next_time <= now:
            next_time += timedelta(days=4)
    
    elif interval == "1w":
        days_until_monday = (7 - now.weekday()) % 7
        if days_until_monday == 0 and now.hour >= 0 and now.minute >= 1:
            days_until_monday = 7
        next_time = now.replace(hour=0, minute=1, second=0, microsecond=0)
        next_time += timedelta(days=days_until_monday)
        while next_time <= now:
            next_time += timedelta(days=7)
    
    else:
        logger.warning(f"Unrecognized interval: {interval}, defaulting to 1d")
        return get_next_candle_time("1d", now=now)
    
    return next_time

def should_run_timeframe_today(timeframe, now=None):
    """
    Check if a timeframe should run today based on aggregation schedule
    
    Args:
        timeframe (str): Timeframe to check
        now (datetime, optional): Mock current time for testing
    """
    now = now or datetime.utcnow()
    
    if timeframe == "1d" or timeframe == "1w":
        # Native timeframes
        if timeframe == "1w":
            return now.weekday() == 0  # Monday
        return True  # Daily runs every day
    
    elif timeframe == "2d":
        reference_date = pd.Timestamp('2025-03-20').normalize()
        today = pd.Timestamp(now.date())
        days_diff = (today - reference_date).days
        return days_diff % 2 == 0
    
    elif timeframe == "3d":
        reference_date = pd.Timestamp('2025-03-20').normalize()
        today = pd.Timestamp(now.date())
        days_diff = (today - reference_date).days
        return days_diff % 3 == 0
    
    elif timeframe == "4d":
        reference_date = pd.Timestamp('2025-03-22').normalize()
        today = pd.Timestamp(now.date())
        days_diff = (today - reference_date).days
        return days_diff % 4 == 0
    
    return False

def get_active_timeframes_for_today(now=None):
    """Get list of timeframes that should run today"""
    all_timeframes = ["1d", "2d", "3d", "4d", "1w"]
    return [tf for tf in all_timeframes if should_run_timeframe_today(tf, now=now)]

# ═════════════════════════════════════════════════════════════════════════════════════════
# Optimized scan execution with prioritization
# ═════════════════════════════════════════════════════════════════════════════════════════

async def run_optimized_scan(config, active_timeframes):
    """Run a single scan configuration with timeframe filtering"""
    try:
        from run_parallel_scanner import run_parallel_multi_timeframes_all_exchanges
        
        # Filter timeframes to only those active today and in config
        scan_timeframes = [tf for tf in config["timeframes"] if tf in active_timeframes]
        
        if not scan_timeframes:
            logger.info(f"Skipping {config['name']} - no active timeframes today")
            return 0
        
        logger.info(f"Running {config['name']} scan for timeframes: {scan_timeframes}")
        logger.info(f"  Strategies: {config['strategies']}")
        logger.info(f"  Exchanges: {config['exchanges']}")
        logger.info(f"  Priority: {config['priority']} ({config['exchange_type']})")
        
        result = await run_parallel_multi_timeframes_all_exchanges(
            timeframes=scan_timeframes,
            strategies=config['strategies'],
            exchanges=config['exchanges'],
            users=config['users'],
            send_telegram=config['send_telegram'],
            min_volume_usd=config['min_volume_usd']
        )
        
        signal_count = sum(len(signals) for signals in result.values())
        logger.info(f"Completed {config['name']}: {signal_count} signals found")
        
        return signal_count
    
    except Exception as e:
        logger.error(f"Error in {config['name']} scan: {str(e)}")
        return 0

async def run_prioritized_scans(active_timeframes):
    """
    Run all scan configurations in priority order with optimized data fetching
    """
    logger.info("═══════════════════════════════════════════════════════════════")
    logger.info(f"Starting prioritized scan session for timeframes: {active_timeframes}")
    logger.info("═══════════════════════════════════════════════════════════════")
    
    # Prepare session data if needed for aggregated timeframes
    await session_manager.prepare_session_data(active_timeframes, all_spot_exchanges + all_futures_exchanges)
    
    # Sort configs by priority for execution order
    sorted_configs = sorted(scan_configs, key=lambda x: x['priority'])
    
    total_signals = 0
    
    # Group configs by priority for potential parallel execution within priority levels
    priority_groups = {}
    for config in sorted_configs:
        priority = config['priority']
        if priority not in priority_groups:
            priority_groups[priority] = []
        priority_groups[priority].append(config)
    
    # Execute each priority group
    for priority in sorted(priority_groups.keys()):
        group_configs = priority_groups[priority]
        logger.info(f"Executing priority {priority} group ({len(group_configs)} configs)")
        
        # Within each priority group, we can run configs in parallel
        # But for now, run sequentially to respect API limits
        group_signals = 0
        for config in group_configs:
            signals = await run_optimized_scan(config, active_timeframes)
            group_signals += signals
            
            # Small delay between configs in same priority group
            await asyncio.sleep(5)
        
        logger.info(f"Priority {priority} complete: {group_signals} signals")
        total_signals += group_signals
        
        # Longer delay between priority groups to let fast exchanges complete
        # before starting slow exchanges
        if priority < max(priority_groups.keys()):
            logger.info("Waiting before next priority group...")
            await asyncio.sleep(15)
    
    # Cache management - clear if session contained aggregated timeframes
    if should_clear_cache_for_session(active_timeframes):
        logger.info("Clearing cache after aggregated timeframes session")
        try:
            kline_cache.clear()
        except:
            pass
        session_manager.clear_session_cache()
    
    logger.info("═══════════════════════════════════════════════════════════════")
    logger.info(f"Session complete: {total_signals} total signals across all priorities")
    logger.info("═══════════════════════════════════════════════════════════════")
    
    return total_signals

# ═════════════════════════════════════════════════════════════════════════════════════════
# Simulation Runner (replaces the infinite scheduler loop)
# ═════════════════════════════════════════════════════════════════════════════════════════

# Mock time: 2025-09-22 00:01:00 UTC (Monday where ALL timeframes are active)
mock_time = datetime(2025, 9, 22, 0, 1, 0)

async def run_simulation():
    """Run a single simulated scan session with mocked time"""
    logger.info("Starting Jupyter simulation of AWS Scanner Service")
    logger.info(f"Mocked time: {mock_time.strftime('%Y-%m-%d %H:%M:%S')} UTC")
    logger.info(f"Strategy Classification:")
    logger.info(f"  Native strategies: {native_strategies}")
    logger.info(f"  Composed strategies: {composed_strategies}")
    logger.info(f"  Futures-only strategies: {futures_only_strategies}")
    logger.info(f"Exchange Classification:")
    logger.info(f"  Fast spot exchanges: {fast_spot_exchanges}")
    logger.info(f"  Fast futures exchanges: {fast_futures_exchanges}")
    logger.info(f"  Slow spot exchanges: {slow_spot_exchanges}")
    logger.info(f"  Slow futures exchanges: {slow_futures_exchanges}")
    logger.info(f"Execution Priority:")
    logger.info(f"  1. Fast Native → 2. Fast Composed → 3. Fast Futures-Only → 4. Slow Native → 5. Slow Composed")
    
    # Get active timeframes under mock time (should be all 5)
    active_timeframes = get_active_timeframes_for_today(now=mock_time)
    logger.info(f"Active timeframes today (mocked): {active_timeframes}")
    
    if len(active_timeframes) != 5:
        logger.warning("Not all timeframes are active - check mock date!")
        return 0
    
    # Clear caches for fresh start
    try:
        kline_cache.clear()
    except:
        pass
    session_manager.clear_session_cache()
    
    # Run the prioritized scans
    total_signals = await run_prioritized_scans(active_timeframes)
    
    logger.info(f"Simulation complete: {total_signals} total signals")
    return total_signals

# Run the simulation
# (In Jupyter, this will execute asynchronously - output will appear in the notebook)
await run_simulation()

2025-09-18 11:22:21 - INFO: Starting Jupyter simulation of AWS Scanner Service
2025-09-18 11:22:21 - INFO: Mocked time: 2025-09-22 00:01:00 UTC
2025-09-18 11:22:21 - INFO: Strategy Classification:
2025-09-18 11:22:21 - INFO:   Native strategies: ['confluence', 'consolidation_breakout', 'channel_breakout', 'loaded_bar', 'trend_breakout', 'pin_up', 'sma50_breakout']
2025-09-18 11:22:21 - INFO:   Composed strategies: ['hbs_breakout', 'vs_wakeup']
2025-09-18 11:22:21 - INFO:   Futures-only strategies: ['reversal_bar', 'pin_down']
2025-09-18 11:22:21 - INFO: Exchange Classification:
2025-09-18 11:22:21 - INFO:   Fast spot exchanges: ['binance_spot', 'bybit_spot', 'gateio_spot']
2025-09-18 11:22:21 - INFO:   Fast futures exchanges: ['binance_futures', 'bybit_futures', 'gateio_futures']
2025-09-18 11:22:21 - INFO:   Slow spot exchanges: ['kucoin_spot', 'mexc_spot']
2025-09-18 11:22:21 - INFO:   Slow futures exchanges: ['mexc_futures']
2025-09-18 11:22:21 - INFO: Execution Priority:
2025-09-18

,symbol,date,close,volume,volume_usd,volume_ratio,close_off_low,momentum_score,high_volume,volume_breakout,extreme_volume,extreme_spread,spread_breakout,momentum_breakout,current_bar,direction,is_engulfing_reversal,timeframe,exchange
111,1000000MOGUSDT,2025-09-17 00:00:00+00:00,1.01840,1.303243e+07,1.327223e+07,1.452630,82.587413,0.378448,True,True,False,False,True,True,False,Up,False,1d,binance_futures
112,AIOUSDT,2025-09-18 00:00:00+00:00,0.19216,1.173780e+08,2.255535e+07,0.701380,84.127306,0.502505,True,False,False,False,True,True,True,Up,False,1d,binance_futures
113,ARCUSDT,2025-09-17 00:00:00+00:00,0.02330,5.503752e+08,1.282374e+07,0.913464,93.700787,0.994229,True,False,False,False,True,True,False,Up,False,1d,binance_futures
114,AVAXUSDT,2025-09-17 00:00:00+00:00,31.82500,2.511073e+07,7.991491e+08,1.021370,93.292914,0.926041,True,False,False,False,True,True,False,Up,False,1d,binance_futures
115,AXLUSDT,2025-09-17 00:00:00+00:00,0.33140,2.250662e+07,7.458693e+06,1.164893,76.315789,0.107069,True,False,False,False,True,True,False,Up,False,1d,binance_futures
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,IMX_USDT,2025-09-14 00:00:00+00:00,0.75270,7.427681e+06,5.590815e+06,2.919697,89.202762,0.509889,True,True,False,False,True,True,False,Up,False,4d,gateio_spot
380,JTO_USDT,2025-09-14 00:00:00+00:00,2.05200,3.855914e+06,7.912335e+06,1.454968,72.826087,0.156705,True,False,False,False,True,True,False,Up,False,4d,gateio_spot
381,KMNO_USDT,2025-09-14 00:00:00+00:00,0.08938,1.639231e+07,1.465145e+06,1.829488,88.521910,0.638198,True,True,False,False,True,True,False,Up,False,4d,gateio_spot
382,NUMI_USDT,2025-09-14 00:00:00+00:00,0.10925,2.414689e+07,2.638047e+06,0.589857,76.192098,0.277667,True,False,False,False,True,True,False,Up,False,4d,gateio_spot


2025-09-18 12:25:25 - INFO: 
Consolidation Breakout: 135 signals


,symbol,date,close,current_bar,volume_usd,volume_ratio,close_position_indicator,close_position_pct,breakout,direction,...,min_bars_inside_req,height_pct,max_height_pct_req,atr_ok,range_high,range_low,range_mid,strength_label,timeframe,exchange
14,TAUSDT,2025-09-18 00:00:00+00:00,0.125160,True,3.884259e+07,0.721951,○○●,79.855247,True,Up,...,4,32.371178,35.0,True,0.135850,0.098000,0.116925,Regular,1d,binance_futures
129,1MBABYDOGEUSDT,2025-09-15 00:00:00+00:00,0.001388,True,4.326302e+07,0.642775,○○●,79.581152,True,Up,...,4,37.991570,35.0,True,0.001623,0.001105,0.001364,Regular,1w,binance_futures
130,ALPHAUSDT,2025-09-15 00:00:00+00:00,0.021300,True,3.113054e+08,3.510337,○●○,39.721254,True,Up,...,4,33.134983,35.0,True,0.018610,0.013320,0.015965,Strong,1w,binance_futures
131,DEXEUSDT,2025-09-15 00:00:00+00:00,8.259000,True,4.772363e+07,1.408121,○○●,76.537137,True,Up,...,4,36.812856,35.0,True,8.842000,6.093000,7.467500,Regular,1w,binance_futures
132,IMXUSDT,2025-09-15 00:00:00+00:00,0.780600,True,3.110489e+08,1.718967,○○●,99.301513,True,Up,...,4,37.523711,35.0,False,0.688700,0.471100,0.579900,Strong,1w,binance_futures
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,DEXE_USDT,2025-09-18 00:00:00+00:00,8.268000,True,5.811037e+06,1.597196,○○●,76.511335,True,Up,...,4,14.168449,15.0,True,7.762000,6.735000,7.248500,Strong,4d,gateio_spot
99,GMX_USDT,2025-09-14 00:00:00+00:00,15.560000,False,1.508732e+07,1.563681,○○●,93.258427,True,Up,...,4,19.556714,25.0,True,16.840000,13.840000,15.340000,Regular,4d,gateio_spot
100,GRASS_USDT,2025-09-18 00:00:00+00:00,0.890500,True,2.450417e+06,0.492689,○○●,77.925785,True,Up,...,4,24.699872,25.0,True,0.879700,0.686300,0.783000,Strong,4d,gateio_spot
101,LA_USDT,2025-09-18 00:00:00+00:00,0.447200,True,2.167380e+06,2.765180,●○○,3.992989,True,Up,...,4,33.933538,35.0,True,0.399500,0.283600,0.341550,Strong,4d,gateio_spot


2025-09-18 12:25:25 - INFO: 
Channel Breakout: 195 signals


...

,symbol,date,close,current_bar,volume_usd,volume_ratio,close_position_indicator,close_position_pct,high,low,bullishbottom_dist,low_wick_ratio,bullish_candle,range_vs_atr,timeframe,exchange
5,HIFIUSDT,2025-09-17 00:00:00+00:00,0.141950,False,1.550148e+09,4.143859,●○○,5.977774,0.677510,0.107900,2,0.090020,False,2.507323,4d,binance_futures
0,BBQUSDT,2025-09-17 00:00:00+00:00,0.002293,False,2.914258e+05,3.923494,○○●,73.503699,0.002687,0.001200,1,0.160297,False,2.582238,1d,bybit_spot
1,TURBOSUSDT,2025-09-17 00:00:00+00:00,0.000916,False,1.018260e+05,3.157109,○○●,88.559755,0.000927,0.000829,1,0.271261,False,1.132540,1d,bybit_spot
2,SUNDOGUSDT,2025-09-18 00:00:00+00:00,0.036610,True,4.456934e+05,0.187172,○●○,68.202765,0.037300,0.035130,3,0.223140,False,0.408663,4d,bybit_spot
6,NOS_USDT,2025-09-08 00:00:00+00:00,0.906400,False,3.770914e+06,5.319988,○●○,55.060178,1.335800,0.380300,3,0.037263,False,4.441235,1w,gateio_spot
3,HIFI_USDT,2025-09-14 00:00:00+00:00,0.141550,False,7.277232e+06,5.060431,●○○,6.014905,0.675000,0.107410,2,0.090239,False,2.563690,4d,gateio_spot
4,NOS_USDT,2025-09-18 00:00:00+00:00,0.828900,True,1.615626e+05,0.211618,○●○,63.325183,0.888900,0.725300,3,0.189437,False,0.799665,4d,gateio_spot


2025-09-18 12:25:25 - INFO: 
Sma50 Breakout: 360 signals


,symbol,date,close,current_bar,volume_usd,volume_ratio,close_position_indicator,close_position_pct,close_price,volume,...,upper_breakout_threshold,atr_threshold_distance,is_clean_breakout,clean_lookback_period,avg_last_n_distance,direction,strong,strength_label,timeframe,exchange
64,1000CATUSDT,2025-09-18 00:00:00+00:00,0.008386,True,8.551194e+06,0.635806,○●○,43.923611,0.008386,1.019699e+09,...,0.008437,0.000254,True,7,-0.000426,Up,True,Strong,1d,binance_futures
65,1000XUSDT,2025-09-18 00:00:00+00:00,0.051240,True,5.983502e+05,0.246158,○●○,63.366337,0.051240,1.167740e+07,...,0.052806,0.000018,True,7,-0.002050,Up,False,,1d,binance_futures
66,ANIMEUSDT,2025-09-18 00:00:00+00:00,0.016350,True,1.827720e+06,0.323224,○●○,65.625000,0.016350,1.117871e+08,...,0.016645,0.000049,True,7,-0.000662,Up,False,,1d,binance_futures
67,BABYUSDT,2025-09-18 00:00:00+00:00,0.060220,True,3.299158e+07,1.721734,○○●,83.209352,0.060220,5.478509e+08,...,0.054545,0.007686,True,7,-0.003098,Up,True,Strong,1d,binance_futures
68,BIGTIMEUSDT,2025-09-18 00:00:00+00:00,0.055810,True,2.526674e+06,0.495751,○○●,89.436620,0.055810,4.527278e+07,...,0.056324,0.000516,True,7,-0.002306,Up,False,Regular,1d,binance_futures
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,SNX_USDT,2025-09-18 00:00:00+00:00,0.695000,True,8.078967e+04,0.085317,●○○,19.166667,0.695000,1.162441e+05,...,0.709817,0.014629,True,7,-0.041294,Up,False,,4d,gateio_spot
288,STRK_USDT,2025-09-18 00:00:00+00:00,0.137680,True,6.084924e+04,0.085377,○●○,42.287234,0.137680,4.419613e+05,...,0.140899,0.002032,True,7,-0.015794,Up,False,,4d,gateio_spot
289,TIMECHRONO_USDT,2025-09-14 00:00:00+00:00,12.013000,False,1.061210e+06,1.112955,●○○,19.542254,12.013000,8.833846e+04,...,12.614220,0.119980,True,7,-1.142244,Up,False,,4d,gateio_spot
290,USDP_USDT,2025-09-18 00:00:00+00:00,0.999700,True,2.569957e+02,0.000168,○●○,50.000000,0.999700,2.570729e+02,...,1.000250,0.000078,True,7,-0.000602,Up,False,,4d,gateio_spot


2025-09-18 12:25:25 - INFO: Completed fast_native_strategies: 1287 signals found
2025-09-18 12:25:30 - INFO: Priority 1 complete: 1287 signals
2025-09-18 12:25:30 - INFO: Waiting before next priority group...
2025-09-18 12:25:45 - INFO: Executing priority 2 group (1 configs)
2025-09-18 12:25:45 - INFO: Running fast_composed_strategies scan for timeframes: ['1d', '2d', '3d', '4d', '1w']
2025-09-18 12:25:45 - INFO:   Strategies: ['hbs_breakout', 'vs_wakeup']
2025-09-18 12:25:45 - INFO:   Exchanges: ['binance_spot', 'bybit_spot', 'gateio_spot', 'binance_futures']
2025-09-18 12:25:45 - INFO:   Priority: 2 (fast_mixed)
2025-09-18 12:25:45 - INFO: 
2025-09-18 12:25:45 - INFO:   RUNNING PARALLEL MULTI-TIMEFRAME SCAN ON ALL EXCHANGES
2025-09-18 12:25:45 - INFO: ================================================================================

2025-09-18 12:25:45 - INFO: • Exchanges: binance_spot, bybit_spot, gateio_spot, binance_futures
2025-09-18 12:25:45 - INFO: • Timeframes: 1d, 2d, 3d, 4d, 

CancelledError: 